In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
import censusgeocode as cg
import time
import os

In [2]:
census_tracts = gp.read_file('/Users/ameliabaum/Desktop/Amelia/Berkeley 2018-2019/EnRes C190/evictionmodel_190final/cb_2017_06_tract_500k/cb_2017_06_tract_500k.shp')
sf_tracts = census_tracts[census_tracts['COUNTYFP'] == '075']
sf_tracts.head() #196 census tracts in SF


DriverError: /Users/ameliabaum/Desktop/Amelia/Berkeley 2018-2019/EnRes C190/evictionmodel_190final/cb_2017_06_tract_500k/cb_2017_06_tract_500k.shp: No such file or directory

In [4]:
sf_evictions = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/Berkeley 2018-2019/EnRes C190/evictionmodel_190final/data/Eviction_Notices.csv")
sf_evictions["Year"] = sf_evictions["File Date"].astype(str).apply(lambda row: row[6:])
sf_evictions["Year"] = sf_evictions["Year"].astype(int)
# sf_evictions_11 = sf_evictions[sf_evictions["Year"] >= 2011].dropna()
# sf_evictions_11 = sf_evictions_11.replace({False:0, True: 1})
# sf_evictions_11.head()
sf_evictions_17 = sf_evictions[sf_evictions["Year"] >= 2017].dropna()
sf_evictions_17 = sf_evictions_17.replace({False:0, True: 1})
sf_evictions_17.head()




/Users/ameliabaum/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Eviction ID,Address,City,State,Eviction Notice Source Zipcode,File Date,Non Payment,Breach,Nuisance,Illegal Use,...,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Constraints Date,Supervisor District,Neighborhoods - Analysis Boundaries,Location,Year
6,M172623,100 Block Of Charles Street,San Francisco,CA,94131,10/19/2017,0,0,0,0,...,0,0,0,0,0,12/17/2022,8.0,Glen Park,"(37.738336951880164, -122.4256608010048)",2017
10,M172628,1300 Block Of Chestnut Street,San Francisco,CA,94123,10/23/2017,0,0,0,0,...,0,0,0,0,0,12/17/2022,2.0,Marina,"(37.80223066530217, -122.42556524759928)",2017
14,M172503,1200 Block Of 41st Avenue,San Francisco,CA,94122,10/02/2017,0,0,0,0,...,0,0,0,0,0,11/29/2022,4.0,Sunset/Parkside,"(37.76346716977734, -122.50069529991866)",2017
22,M172511,0 Block Of 5th Avenue,San Francisco,CA,94118,10/10/2017,0,0,0,0,...,0,0,0,0,0,12/05/2022,2.0,Inner Richmond,"(37.787705866671075, -122.46361972068956)",2017
25,M172709,100 Block Of Prospect Avenue,San Francisco,CA,94110,10/26/2017,0,0,0,0,...,0,0,0,0,0,12/24/2022,9.0,Bernal Heights,"(37.74296049255459, -122.41894065545237)",2017


In [9]:
#split the lat and lon into different columns
# sf_evictions_11["Latitude"] = sf_evictions_11["Location"].astype(str).apply(lambda row: row.split(",")[0][1:])
# sf_evictions_11["Longitude"] = sf_evictions_11["Location"].astype(str).apply(lambda row: row.split(",")[1][:-1])

sf_evictions_17["Latitude"] = sf_evictions_17["Location"].astype(str).apply(lambda row: row.split(",")[0][1:])
sf_evictions_17["Longitude"] = sf_evictions_17["Location"].astype(str).apply(lambda row: row.split(",")[1][:-1])
len(sf_evictions_17)

447

In [6]:
def getTract(lon, lat):
#pings the FCC API to geocode lat lon coordinates to other geograpies. Returns a 2 item array of [geoid, tract name]
    try:
        result = cg.coordinates(lon, lat)
        tract = result['Census Tracts'][0]['NAME'] #alt use 'TRACT' for partial fips
        return tract
    except(ValueError, TypeError):
        return 'Nan'


In [7]:
def getGeoid(lon, lat):
#pings the FCC API to geocode lat lon coordinates to other geograpies. Returns a 2 item array of [geoid, tract name]
    try:
        result = cg.coordinates(lon, lat)
        geoid = result['Census Tracts'][0]['GEOID']
        return geoid
    except(ValueError, TypeError):
        return 'Nan'
        

In [10]:
start = time.time()
#sf_evictions_11.head()["Geoid"] = sf_evictions_11.head().apply(lambda row: getGeoid(row["Longitude"], row["Latitude"]), axis=1)
sf_evictions_17["Geoid"] = sf_evictions_17.apply(lambda row: getGeoid(row["Longitude"], row["Latitude"]), axis=1)

end = time.time()
elapsed = end-start #this takes like ~30 mins



In [13]:
sf_evictions_17.head()

,Eviction ID,Address,City,State,Eviction Notice Source Zipcode,File Date,Non Payment,Breach,Nuisance,Illegal Use,...,Development,Good Samaritan Ends,Constraints Date,Supervisor District,Neighborhoods - Analysis Boundaries,Location,Year,Latitude,Longitude,Geoid
6,M172623,100 Block Of Charles Street,San Francisco,CA,94131,10/19/2017,0,0,0,0,...,0,0,12/17/2022,8.0,Glen Park,"(37.738336951880164, -122.4256608010048)",2017,37.738336951880164,-122.4256608010048,06075021800
10,M172628,1300 Block Of Chestnut Street,San Francisco,CA,94123,10/23/2017,0,0,0,0,...,0,0,12/17/2022,2.0,Marina,"(37.80223066530217, -122.42556524759928)",2017,37.80223066530217,-122.42556524759928,06075012901
14,M172503,1200 Block Of 41st Avenue,San Francisco,CA,94122,10/02/2017,0,0,0,0,...,0,0,11/29/2022,4.0,Sunset/Parkside,"(37.76346716977734, -122.50069529991866)",2017,37.76346716977734,-122.50069529991866,06075035100
22,M172511,0 Block Of 5th Avenue,San Francisco,CA,94118,10/10/2017,0,0,0,0,...,0,0,12/05/2022,2.0,Inner Richmond,"(37.787705866671075, -122.46361972068956)",2017,37.787705866671075,-122.46361972068956,06075040100
25,M172709,100 Block Of Prospect Avenue,San Francisco,CA,94110,10/26/2017,0,0,0,0,...,0,0,12/24/2022,9.0,Bernal Heights,"(37.74296049255459, -122.41894065545237)",2017,37.74296049255459,-122.41894065545237,Nan


In [14]:
# sf_evictions_11.to_csv("data/evictions_geocoded_11.csv")
sf_evictions_17.to_csv("data/evictions_geocoded_17.csv")

In [406]:
# evictions_geocoded11_tractandyear = sf_evictions_11.groupby(["Geoid", 
#         "Year"])
# evictions_geocoded11_tractandyear
# .drop(["Neighborhoods - Analysis Boundaries", "Location", "Latitude", "Longitude", 
#         "Eviction Notice Source Zipcode", "Address", "City", "State", "Constraints Date", "File Date", "Supervisor District"], axis=1) #166 census tracts represented


In [31]:
evictions_geocoded17_tractandyear = sf_evictions_17.drop(["Neighborhoods - Analysis Boundaries", "Location", "Latitude", "Longitude", 
        "Eviction Notice Source Zipcode", "Address", "City", "State", "Constraints Date", "File Date", "Supervisor District"], axis=1) #166 census tracts represented


evictions_geocoded17_tractandyear.head()

,Eviction ID,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Year,Geoid
6,M172623,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,2017,06075021800
10,M172628,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,2017,06075012901
14,M172503,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,2017,06075035100
22,M172511,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,2017,06075040100
25,M172709,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,2017,Nan


In [44]:
evictions_geocoded17_tractandyear = evictions_geocoded17_tractandyear.groupby(["Geoid", "Year"]).sum()

In [49]:
evictions_geocoded17_tractandyear.sum(axis=0)

Non Payment                  0
Breach                       0
Nuisance                     0
Illegal Use                  0
Failure to Sign Renewal      0
Access Denial                0
Unapproved Subtenant         0
Owner Move In              447
Demolition                   0
Capital Improvement          0
Substantial Rehab            0
Ellis Act WithDrawal         0
Condo Conversion             0
Roommate Same Unit           0
Other Cause                  0
Late Payments                0
Lead Remediation             0
Development                  0
Good Samaritan Ends          0
dtype: int64

In [50]:
#evictions_geocoded11_tractandyear.to_csv("data/evictions_geocoded11_tractandyear.csv")
evictions_geocoded17_tractandyear.to_csv("data/evictions_geocoded17_tractandyear.csv" )

In [11]:
# pd.read_csv("data/evictions_geocoded11_tractandyear.csv")

In [295]:
# evictions_geocoded11_tractandyear = pd.read_csv("data/evictions_geocoded11_tractandyear.csv")
# evictions_geocoded11_tractandyear["Tract"] = evictions_geocoded11_tractandyear["Geoid"].astype(str).apply(lambda row: row[5:])
# evictions_geocoded11_tractandyear["tract_year"] = evictions_geocoded11_tractandyear["Tract"] + evictions_geocoded11_tractandyear["Year"].astype(str)
# evictions_geocoded11_tractandyear.to_csv("data/evictions_geocoded11_tractandyear.csv")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
                                                                                                          
                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
evictions_geocoded17_tractandyear["Tract"] = evictions_geocoded17_tractandyear["Geoid"].astype(str).apply(lambda row: row[5:])
evictions_geocoded17_tractandyear["tract_year"] = evictions_geocoded17_tractandyear["Tract"] + evictions_geocoded17_tractandyear["Year"].astype(str)
evictions_geocoded17_tractandyear.to_csv("data/evictions_geocoded17_tractandyear.csv")       

In [12]:
# data_path = '/Users/ameliabaum/Desktop/Amelia/Berkeley 2018-2019/EnRes C190/evictionmodel_190final/data/'
# #table B02001 race
# #race_feature = {}
# race_feature = []
# for race_file in os.listdir(data_path+'B02001_race'):
#     year = race_file.split("_")[1]
#     #if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
#     if year in ['09', '10', '11', '12', '13', '14', '15', '16']:

#     #race_feature['20'+year] = pd.read_csv(data_path+'B02001_race/'+race_file)
#     df = pd.read_csv(data_path+'B02001_race/'+race_file)
#     race_feature.append(df)
# # print(race_feature.keys())
# print(len(race_feature))


# #table B25071 rent burden
# burden_feature = {}
# for burden_file in os.listdir(data_path+'B25071_burden'):
#     year = burden_file.split("_")[1]
#     if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
#         df = pd.read_csv(data_path+'B25071_burden/'+burden_file)
#         burden_feature['20'+year] = df
        
# #print(burden_feature.keys())

# #table S1101 household size
# hsize_feature = {}
# for size_file in os.listdir(data_path+'S1101_size'):
#     year = size_file.split("_")[1]
#     if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
#         hsize_feature['20'+year] = pd.read_csv(data_path+'S1101_size/'+size_file)
# #print(hsize_feature.keys())

# #table S1501 % with bachelors degree
# edu_feature = {}
# for edu_file in os.listdir(data_path+'S1501_edu'):
#     year = edu_file.split("_")[1]
#     if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
#         edu_feature['20'+year] = pd.read_csv(data_path+'S1501_edu/'+edu_file)
# #print(edu_feature.keys())

# #table B02001 race
# race_feature = {}
# for race_file in os.listdir(data_path+'B02001_race'):
#     year = race_file.split("_")[1]
#     if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
#         race_feature['20'+year] = pd.read_csv(data_path+'B02001_race/'+race_file)
# #print(race_feature.keys())



# #table S1101 household size
# size_feature = {}
# for size_file in os.listdir(data_path+'S1101_size'):
#     year = size_file.split("_")[1]
#     if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
#         size_feature['20'+year] = pd.read_csv(data_path+'S1101_size/'+size_file)
# #print(size_feature.keys())

# #table B25035 median year built
# built_feature = {}
# for built_file in os.listdir(data_path+'B25035_yearbuilt'):
#     year = built_file.split("_")[1]
#     if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
#         built_feature['20'+year] = pd.read_csv(data_path+'B25035_yearbuilt/'+built_file)
# #print(built_feature.keys())


# #table B25039 median renter moved in
# moved_feature = {}
# for moved_file in os.listdir(data_path+'B25039_moved'):
#     year = moved_file.split("_")[1]
#     if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
#         moved_feature['20'+year] = pd.read_csv(data_path+'B25039_moved/'+moved_file)
# #print(moved_feature.keys())


# #table B25001 housing units in tract
# units_feature = {}
# for units_file in os.listdir(data_path+'B25001_units'):
#     year = units_file.split("_")[1]
#     if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
# #         foo = pd.read_csv(data_path+'B25001_units/'+units_file)
# #         print(foo.dtypes)
#         units_feature['20'+year] = pd.read_csv(data_path+'B25001_units/'+units_file)
# #print(units_feature.keys())




In [386]:
#use groupby to group the evictions data by year, and then perform in inner merge with the feature
evictions = pd.read_csv("data/evictions_geocoded11_tractandyear.csv").drop("Unnamed: 0", axis=1)
total = evictions[['Non Payment', 'Breach', 'Nuisance',
       'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends']]
evictions["Total Evictions"] = total.sum(axis=1)
total_evictionsbytract = evictions[["Geoid", "Year", "Tract", "tract_year", "Total Evictions"]].dropna()
total_evictionsbytract["Geoid"] = total_evictionsbytract["Geoid"].astype(int)
evictions

,Geoid,Year,Eviction ID,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,...,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Tract,tract_year,Total Evictions
0,6075010100,2011,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,10100.0,101002011,38
1,6075010200,2012,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,10200.0,102002012,19
2,6075010200,2017,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,10200.0,102002017,19
3,6075010300,2012,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,10300.0,103002012,38
4,6075010300,2013,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,10300.0,103002013,38
5,6075010300,2014,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,10300.0,103002014,57
6,6075010300,2015,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,10300.0,103002015,76
7,6075010300,2016,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,10300.0,103002016,19
8,6075010300,2017,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,10300.0,103002017,19
9,6075010300,2018,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,10300.0,103002018,38
